## Marine Species Image Classifier (Part 2 Test)

In [ ]:
# loading images and their predictions 
from google.cloud import storage
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np
from PIL import Image
import os

# import cv2

# Initialize GCS client
storage_client = storage.Client()
bucket_name = 'marine-science-image-bucket'  # Replace with your GCS bucket name
bucket = storage_client.get_bucket(bucket_name)

ORIGINAL_TEST_DIR = 'marine-science-image-bucket/images/test/'  # Replace with your test directory path in the bucket
IMAGE_SIZE = 256  # Replace with your desired image size
labels = ['class1', 'class2', 'class3']  # Replace with your actual class labels

prediction = []
original = []
image = []
count = 0

# List blobs in the GCS bucket
blobs = storage_client.list_blobs(bucket_name, prefix=ORIGINAL_TEST_DIR)

for blob in blobs:
    if blob.name.endswith('/'):
        continue  # Skip directories

    # Extract the class name from the blob path
    class_name = os.path.basename(os.path.dirname(blob.name))
    
    # Download the blob to a temporary file
    blob_bytes = blob.download_as_bytes()
    
    # Open the image using PIL
    img = Image.open(io.BytesIO(blob_bytes))
    
    # Resize the image
    img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
    
    # Append image to the image list
    image.append(img)
    
    # Convert image to array
    img = np.asarray(img, dtype=np.float32)
    
    # Normalize the image
    img = img / 255
    
    # Reshape the image into a 4D array
    img = img.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
    
    # Make prediction using the model
    predict = model.predict(img)
    
    # Get the index corresponding to the highest value in the prediction
    predict = np.argmax(predict)
    
    # Append the predicted class to the list
    prediction.append(labels[predict])
    
    # Append the original class to the list
    original.append(class_name)

In [ ]:
# visualizing the results
import random
fig=plt.figure(figsize = (25,30))
for i in range(20):
    j = random.randint(0,len(image))
    fig.add_subplot(5,4,i+1)
    fig.suptitle('Test Predictions', fontsize=40)
    plt.xlabel("Prediction -" + prediction[j] +"   Original -" + original[j] , fontsize=22)
    plt.imshow(image[j])
fig.tight_layout()

In [ ]:
# plot confusion matrix

plt.figure(figsize=(20,10))

display_labels=['Fish', 'Goldfish', 'HarborSeal', 'Jellyfish', 'Lobster', 'Oyster', 'SeaTurtle', 'Squid', 'Starfish']
cm = confusion_matrix(np.asarray(original), np.asarray(prediction))
ax = plt.subplot()
sns.heatmap(cm, annot = True, ax = ax , xticklabels=display_labels, yticklabels=display_labels)

ax.set_xlabel('Predicted')
ax.set_ylabel('Original')
ax.set_title('Confusion_matrix')